In [1]:
import numpy as np
import warnings
from sklearn import decomposition

#suppress warnings
warnings.filterwarnings('ignore')

# BLG527E - Machine Learning, Homework 3

In this homework, you are supposed to implement following parts:
     
- **Part 1: solve an SVM optimization problem by hand (50 points)**

- **Part 2: Federate a logistic regression classifier (40 points)**

- **Part 3: practice feature selection (10 points)**
     - Refer to Machine Learning Blinks 10 and 11 for this part:
     - ML Blinks 10: https://www.youtube.com/watch?v=laeth5oT9YM&list=PLug43ldmRSo1LDlvQOPzgoJ6wKnfmzimQ
     - ML Blinks 11: https://www.youtube.com/watch?v=mRmVKNklE9I&list=PLug43ldmRSo1LDlvQOPzgoJ6wKnfmzimQ 
 
 
 
 ### Important Notes:
   - Please complete this template and include any other necessary materials (screenshots of your handwritten solutions etc.) into the HW3 folder. Then zip it again and submit to Ninova.
   - At Part 1, you can upload the screenshots of your handwritten solutions to the Notebook. But please be sure that your solutions are neat and can be read properly.
   - For the Part 2, you will Federate a classifier across 20 datasets. Check the paper in Part 2.2 for more details. 
   - For the Part 3, you should implement feature selection from scratch however can use scikit-learn built-in functions for training the SVM.
   - You can ask your questions via kamard@itu.edu.tr 
    



## Part 1: Solving SVM optimization by hand (50 points)

You can insert the screenshots of your handwritten solution on Jupyter Notebook. For an example, check the cells including images. Do not forget to include your solution image file into the submitted .zip file.

Some reminders for the question:

 - Lagrangian to optimize: $\mathcal{L}_{primal} = \sum_{i=1}^{n} a_{i} - \frac{1}{2} [\sum_{i=1}^{n}\sum_{j=1}^{n}\alpha_{i}\alpha_{j}y_{i}y_{j}x_{i}^{T}x^{j}] $ 


- Constraint: $\sum_{i=1}^{n} \alpha_{i} y_{i} = 0$


- Optimal parameter: $w^{*} = \sum_{i=1}^{n} \alpha_{i} y_{i} x^{i}$

### Part 1.1 (25 points)

<p style="float: left;"><img src="images/Part1-1.png" width = "200"></p>
        
        Given the two following training samples (n=2), provide below a step-by-step solution
        to estimate the optimal parameters (w and b) of the hyperplane separating the two classes.

### Part 1.1 Solution

*Double click here to insert your solution.*
<img src="" width = "200">

<comment> check with builtin function, draw the classifier, add the picture </comment>

In [2]:
# TODO: delete this part
"""
import matplotlib.pyplot as plt

from sklearn import svm

X = [[2, 3], [2, -2]]
y = [1, -1]
clf = svm.SVC(kernel="linear")
clf.fit(X, y)

print(clf.support_vectors_)
print(clf.support_)
"""

'\nimport matplotlib.pyplot as plt\n\nfrom sklearn import svm\n\nX = [[2, 3], [2, -2]]\ny = [1, -1]\nclf = svm.SVC(kernel="linear")\nclf.fit(X, y)\n\nprint(clf.support_vectors_)\nprint(clf.support_)\n'

### Part 1.2 (15 points) 
If we add a third training point $x_3 = \left [\begin{matrix} -4 \\ 4 \end{matrix}\right] $, will that impact the hyperplane estimated using points $x_1$ and $x_2$? Answer this considering two cases where label for $x_3$ is (1) $y_3 = +1 $, (2) $y_3 = -1 $ and justify. You do not need to solve Lagrangian again, justify your answer drawing the hyperplanes and giving explanations.

### Part 1.2 Solution

*Double click here to insert your solution.*
<img src="" width = "200">

### Part 1.3 (10 points)
Explain how to classify the point $x_{test} = \left [\begin{matrix} -4 \\ -3 \end{matrix}\right] $ using the estimated model at Part 1.1. What is the predicted label of $x_{test}$? 

### Part 1.3 Solution

*Double click here to insert your solution.*
<img src="" width = "200">

## Part 2: Federate a logistic regression classifier across 20 datasets (40 points)

We set out to classify two digits from the MNIST dataset: digit 0 to which we assign the label 0 and digit 1 to which we assign the label 1 using the code of the logistic regression based on the softmax loss. You are allowed to use built-in functions from scikit-learn in the specified parts below, which you can install via one of the following commands (depending on which one you use):

        > python3 -m pip install scikit-learn
        > conda install -c conda-forge scikit-learn

We have 20 local datasets distributed across 20 clients and 1 global test dataset left for testing. The function **parse_mnist_into_clients** in the following cell creates the datasets from the shared files. You do NOT need to change the function. You can simply run the following cell and it will load the datasets for you!

In [3]:
def parse_mnist_into_clients():
    '''
    Parses the MNIST data files into 20 local datasets for clients and 1 global test dataset. 
    Do NOT change this function!
    
    
    Outputs:
    
    local_datasets_data: The MNIST data of the local 20 dataset
    local_datasets_label: Labels of the 20 local dataset. local_datasets_data[i] corresponds to local_datasets_label[i]
    test_data : Global Test Data
    test_label: Labels of the global test data
    '''
    data = np.load('mnist_data.npy')
    labels = np.load('mnist_labels.npy')
    local_datasets_data  = []
    local_datasets_label = []
    
    for i in range(20):
        local_datasets_data.append(np.copy(data[i*703:(i+1)*703]))
        local_datasets_label.append(np.copy(labels[i*703:(i+1)*703]))
    
    test_data = np.copy(data[14060:])  
    test_labels = np.copy(labels[14060:])
    
    return local_datasets_data, local_datasets_label, test_data, test_labels

# Load the data
client_data, client_label, test_data, test_label = parse_mnist_into_clients()
print(test_data)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


A simple example code of the logistic classifier using softmax loss is included below. Use this code to federate the learning of the 20 clients and test their local models on the left-out test dataset. Note that the code below is a very simple logistic regression code and the hyper parameters is not optimized. Feel free to modify it depending on what you need.

In [4]:
def sigmoid(x):
    return 1/ (1 + np.exp(-x))
    
def softmax_grad(X,y):
    # Initializations
    w = np.random.randn(X.shape[1],1);        # random initial point # 784
    alpha = 10**-2
    max_its = 10 #2000
    
    for k in range(max_its):
        grad = calculate_grad(X,y,w) # gradient of the loss function wrt to w
        grad = np.mean(grad, axis=1, keepdims=True) # stochastic gradient descent
        w = w - alpha*grad # update rule of gradient descent
        
        """
        s = sigmoid(-y * (X.T @ w)) # sigmoid(-y * (X.T @ w))
        grad_w = -X @ (s * y) # -X @ (s * y)
        #grad_w = np.sum((s*y) * X.T, 0, keepdims=True)
        
        print(f"grad_w {grad_w.shape}")
        print(f"k {k} w {w.shape}")
        #grad_w = np.sum(grad_w)
        #print(f"grad_w {grad_w.shape}")
        w = w - alpha * grad_w
        """
        
    return w

def calculate_grad(X,y,w): # gradient of the loss function least square cost by using sigmoid(xw + b)
    pred = sigmoid(np.dot(X, w))
    error = pred - y
    temp = np.multiply(error, pred)
    temp = np.multiply(temp, (1 - pred))
    temp = np.dot(X.T, temp)
    grad = 2 * temp
    return grad

def client_update(k, w):
    E = 10
    alpha = alpha = 10**-2
    
    for epoch in range(E):
        grad = calculate_grad(X,y,w) # gradient of the loss function wrt to w
        grad = np.mean(grad, axis=1, keepdims=True) # stochastic gradient descent
        w = w - alpha*grad # update rule of gradient descent
        
    return w

### Part 2.1 (10 points)

Use built-in PCA from scikit-learn library to reduce the dimensionality of the training and testing samples to only two dimension (PC1 and PC2). PS: Watch <a href='https://www.youtube.com/watch?v=mRmVKNklE9I&list=PLug43ldmRSo1LDlvQOPzgoJ6wKnfmzimQ&index=45'>lecture 11</a> (beware of the eavesdropping phenomenon!). Note also that PCA should be applied to each client independently. In federated learning, clients never share data.


In [5]:
def apply_pca(data):
    pca = decomposition.PCA(n_components=2)
    pca.fit(data)
    reduced_data = pca.transform(data)
    return reduced_data

# for training
client_data_reduced = []
for X in client_data:
    X_reduced = apply_pca(X)
    client_data_reduced.append(X_reduced)
    
# for testing #TODO: should be the testing data independent of training
test_data_reduced = apply_pca(test_data)

### Part 2.2 (20 points)

From now on we will use the reduced 2-dimensional training and test datasets. Train a logistic classifier on the reduced 2-dimensional dataset for each client, independently, send the model weights to the server, aggregate and broadcast back to all clients.

Basically, implement the **FederatedAveraging** algorithm in **(https://arxiv.org/pdf/1602.05629.pdf)** for B = infinity (i.e., each client will be trained on all its local training samples). 


In [9]:
client_weights = np.zeros((len(client_data_reduced), client_data_reduced[0].shape[1], 1)) # 20, 2, 1
i = 0
for X, y in zip(client_data_reduced, client_label):
    w = softmax_grad(X, y)
    client_weights[i] = w
    i += 1

# send the model weights to the server, aggregate
server_mean_weights = np.mean(client_weights, axis=0)
print(server_mean_weights)

[[ 1.15058697]
 [-0.15217407]]


### Part 2.3 (10 points)

Test the global (aggregated model) on the left-out test dataset. Plot the test classification accuracy on the global dataset against the number of communication rounds for E = 1, 5 and 10. You might need to use a small number of communication rounds if the convergence is fast. Note that B is always set to infinity. Check out Figure 2 in the paper for more details. What do you notice? Discuss your results.

In [7]:
#Your code here

## Part 3: Feature selection using Variance Threshold (10 points)

In this part, we will use scikit-learn library for training SVM. To remind you, you can install the scikit-learn using following commands:

        > python3 -m pip install scikit-learn
        > conda install -c conda-forge scikit-learn
        
There are lots of machine learning techniques that are available in scikit-learn library. In this problem we will use the **SVM** classifier with linear kernel and implement Variance Threshold feature selection method from scratch.
   

You can check the documentations on the internet to learn how to use SVM classifier and which parameters to use. Necessary functions are imported below.

In [8]:
# This file is associated with the book
# "Machine Learning Refined", Cambridge University Press, 2016.
# by Jeremy Watt, Reza Borhani, and Aggelos Katsaggelos.

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.svm import SVC
import sklearn.datasets as ds

from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'matplotlib'

We will implement feature selection on handwritten digits dataset. In the following cell we load and examine dataset. As you can see the samples are 8 by 8 images where in each row of X, the images are kept as flatten vectors. Each feature represents a pixel (i.e 0th feature is (0, 0) pixel and 8th feature is (1, 0) pixel.

In [ ]:
digit_data = ds.load_digits()
print("Number of samples: ", digit_data.data.shape[0])
print("Number of attributes: ", digit_data.data.shape[1])
print("Classes: ", digit_data.target_names)

c = digit_data.images[0]
for i in range(1, 10):
    c = np.concatenate((c, digit_data.images[i]), 1)

plt.gray() 
plt.matshow(c)
plt.show()

Then we split the data into training and testing sets in order to train the models on training set and test them on unseen test set. Do not change the random state so we will get the same train and test splits.

In [ ]:
X, y = digit_data.data, digit_data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle = True, random_state=20)

print("Number of training samples: ", X_train.shape[0])
print("Number of testing samples: ", X_test.shape[0])

 In the following cell, you should complete the necessary train and test functions for SVM. Fill the specified parts only. Calculate the accuracy score as:

<br>
<center> $ Accuracy = \frac{\#correct \ predictions}{\#total \ samples} $ </center>
<br>

In [ ]:
def train(X, y):
    
    classifier = ...  # define the classifier
    classifier = ...  # fit the classifier on training set
    
    preds = ...  # predict the labels for training set
    
    train_accuracy = ...  # calculate the accuracy (do not use built-in function)
    
    return classifier, train_accuracy

def test(classifier, X, y):
    
    test_accuracy = ...  # calculate the accuracy on test set
    
    return test_accuracy

 Next, use these function to train and test an SVM without feature selection.

In [ ]:
# SVM classifier without feature selection
svm, train_acc = ... # call train function with necessary parameters
test_acc = ...  # call test function with necessary parameters

print("Train acc without feature selection: ", train_acc)
print("Test acc without feature selection: ", test_acc)

 Now, implement Variance Threshold feature selection method on the dataset. The Variance Threshold basicly follows these steps:
  - Calculate the variances of each feature in the dataset. (Hint: You can use numpy for this)
  - Eliminate the features that has variance less then the given threshold.

In [ ]:
##TODO: YOUR CODE GOES HERE##

def variance_threshold(X, threshold=0):
    
    selected = None # keep the indexes of selected feature
    eliminated = None # keep the indexes of eliminated features
    X_reduced = None # X with reduced feature set
    
    #calculate variances of each feature and eliminate features with variance lower than threshold.
    
    
    
    print("Indexes of the eliminated features: ", eliminated) # print the indexes of eliminated features.
    
    print("Number of features for threshold = ", threshold, "is :", X_reduced.shape[1])
    return X_reduced, selected


 Now, call the variance threshold function for **5** distinct threshold values and obtain **different set of features**. Then, train and test an SVM classifier on each set. Do not forget to apply the feature selection on the test set as well.

In [ ]:
##TODO: YOUR CODE GOES HERE##

thresholds = [...]

for th in thresholds:
    print("Threshold = ", th)
    X_reduced, selected = variance_threshold(...)
    X_reduced_test = ...

    svm, train_acc = ... # call train function with necessary parameters
    test_acc = ...  # call test function with necessary parameters

    print("Train acc with feature selection: ", train_acc)
    print("Test acc with feature selection: ", test_acc)


**Bonus Question (5 points)**
Why the model performance varies with the number of selected features? Justify your answer with a toy example (i.e., using a simulated dataset). You can also refer to the results you got in Part 2. 

*Double click here to insert your answer*